<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf

df = pd.read_csv('./tf_tutorial_final.csv', encoding='utf-8')
print(df.shape)
df.head()

(2920, 15)


,datetime,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,y
0,2009-01-01 01:00:00,996.50,265.38,-8.78,94.4,3.33,3.14,0.19,1.96,3.15,1307.86,0.21,0.63,192.7,-8.05
1,2009-01-02 01:00:00,999.34,268.77,-5.19,94.4,4.39,4.15,0.25,2.58,4.15,1293.47,0.28,1.00,148.1,-4.44
2,2009-01-03 01:00:00,1002.39,267.70,-6.35,92.1,4.12,3.79,0.33,2.36,3.78,1301.68,0.27,1.00,292.8,-5.28
3,2009-01-04 01:00:00,992.19,272.35,-5.61,73.0,5.50,4.02,1.49,2.52,4.05,1270.01,2.77,4.75,232.1,-1.42
4,2009-01-05 01:00:00,984.74,272.79,-2.58,92.8,5.44,5.05,0.39,3.20,5.13,1260.66,2.55,4.75,245.3,-1.57


In [58]:
print(tf.__version__)

2.7.0


In [41]:
df['datetime'] = df['datetime'].str.replace('-','')
df['datetime'] = df['datetime'].str.split(' ', expand=True)[0]
df['datetime'] = df['datetime'].astype(int)
df.dtypes

datetime      int64
x1          float64
x2          float64
x3          float64
x4          float64
x5          float64
x6          float64
x7          float64
x8          float64
x9          float64
x10         float64
x11         float64
x12         float64
x13         float64
y           float64
dtype: object

In [42]:
df.head()

,datetime,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,y
0,20090101,996.50,265.38,-8.78,94.4,3.33,3.14,0.19,1.96,3.15,1307.86,0.21,0.63,192.7,-8.05
1,20090102,999.34,268.77,-5.19,94.4,4.39,4.15,0.25,2.58,4.15,1293.47,0.28,1.00,148.1,-4.44
2,20090103,1002.39,267.70,-6.35,92.1,4.12,3.79,0.33,2.36,3.78,1301.68,0.27,1.00,292.8,-5.28
3,20090104,992.19,272.35,-5.61,73.0,5.50,4.02,1.49,2.52,4.05,1270.01,2.77,4.75,232.1,-1.42
4,20090105,984.74,272.79,-2.58,92.8,5.44,5.05,0.39,3.20,5.13,1260.66,2.55,4.75,245.3,-1.57


In [47]:
# 숫자 타입 아닌 데이터 타입 상태로 tf.dataset 못 만듬
X_train, y_train = df.iloc[:, :-1].values, df.iloc[:, [-1]].values
X_train = X_train.astype(int)
y_train = y_train.astype(int)
print(X_train.shape, type(X_train))
print(y_train.shape, type(y_train))

(2920, 14) <class 'numpy.ndarray'>
(2920, 1) <class 'numpy.ndarray'>


In [113]:
# 로그 레벨 3으로 변경해서 경고 표시 안뜨게 하
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


# Test
X_window_size = 3  # 총 7개의 window(총 7개의 row)
stride = 1         # 이전 window와 다음 window 시작점 간의 간격
shift = 3          # 7개의 window를 A, 다음 7개의 window를 B라고 할때, A window날짜와 B window날짜 간의 간격
batch_size = 1

ds_x = tf.data.Dataset.from_tensor_slices(X_train)
print(type(ds_x))
ds_x = ds_x.window(size=X_window_size, stride=stride, shift=shift, drop_remainder=True)
ds_x = ds_x.flat_map(lambda x: x.batch(7))

for x in ds_x.take(4):
    print(x.shape)
    print(x)
    print('-'*100)


<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>
(3, 14)
tf.Tensor(
[[20090101      996      265       -8       94        3        3        0
         1        3     1307        0        0      192]
 [20090102      999      268       -5       94        4        4        0
         2        4     1293        0        1      148]
 [20090103     1002      267       -6       92        4        3        0
         2        3     1301        0        1      292]], shape=(3, 14), dtype=int64)
----------------------------------------------------------------------------------------------------
(3, 14)
tf.Tensor(
[[20090104      992      272       -5       73        5        4        1
         2        4     1270        2        4      232]
 [20090105      984      272       -2       92        5        5        0
         3        5     1260        2        4      245]
 [20090106      997      259      -15       83        2        1        0
         1        1     1336      

In [110]:
# 로그 레벨 3으로 변경해서 경고 표시 안뜨게 하기
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


# Test
X_window_size = 2  # 총 7개의 window(총 7개의 row)
stride = 2         # 이전 window와 다음 window 시작점 간의 간격
shift = 2          # 7개의 window를 A, 다음 7개의 window를 B라고 할때, A window날짜와 B window날짜 간의 간격
batch_size = 1

ds_x = tf.data.Dataset.from_tensor_slices(X_train)
ds_x = ds_x.window(size=X_window_size, stride=stride, shift=shift, drop_remainder=True)
ds_x = ds_x.flat_map(lambda x: x.batch(X_window_size))

for x in ds_x.take(4):
    print(x.shape)
    print(x)
    print('-'*100)


(2, 14)
tf.Tensor(
[[20090101      996      265       -8       94        3        3        0
         1        3     1307        0        0      192]
 [20090103     1002      267       -6       92        4        3        0
         2        3     1301        0        1      292]], shape=(2, 14), dtype=int64)
----------------------------------------------------------------------------------------------------
(2, 14)
tf.Tensor(
[[20090103     1002      267       -6       92        4        3        0
         2        3     1301        0        1      292]
 [20090105      984      272       -2       92        5        5        0
         3        5     1260        2        4      245]], shape=(2, 14), dtype=int64)
----------------------------------------------------------------------------------------------------
(2, 14)
tf.Tensor(
[[20090105      984      272       -2       92        5        5        0
         3        5     1260        2        4      245]
 [20090107      994      2

In [98]:
print(df.shape)
df.head(5)

(2920, 15)


,datetime,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,y
0,20090101,996.50,265.38,-8.78,94.4,3.33,3.14,0.19,1.96,3.15,1307.86,0.21,0.63,192.7,-8.05
1,20090102,999.34,268.77,-5.19,94.4,4.39,4.15,0.25,2.58,4.15,1293.47,0.28,1.00,148.1,-4.44
2,20090103,1002.39,267.70,-6.35,92.1,4.12,3.79,0.33,2.36,3.78,1301.68,0.27,1.00,292.8,-5.28
3,20090104,992.19,272.35,-5.61,73.0,5.50,4.02,1.49,2.52,4.05,1270.01,2.77,4.75,232.1,-1.42
4,20090105,984.74,272.79,-2.58,92.8,5.44,5.05,0.39,3.20,5.13,1260.66,2.55,4.75,245.3,-1.57


In [10]:
X_window_size = 7
y_window_size = 7
stride = 1
shift = 7
batch_size = 1

def window_dataset(X, y, X_window_size, y_window_size, stride, shift, batch_size):
    """
    X:
    y:
    X_window_size:
    y_window_size:
    stride:
    shift:
    batch_size:
    """
    ds_x = tf.data.Dataset.from_tensor_slices(X)
    ds_x = ds_x.window(size=X_window_size, stride=stride, shift=X_window_size, drop_remainder=True)
    ds_x = ds_x.flat_map(lambda x: x.batch(X_window_size))
    
    ds_y = tf.data.Dataset.from_tensor_slices(y)
    ds_y = ds_y.window(size=y_window_size, stride=stride, shift=y_window_size, drop_remainder=True)
    ds_y = ds_y.flat_map(lambda y: y.batch(y_window_size))
    
    ds = tf.data.Dataset.zip((ds_x, ds_y))
    return ds.batch(batch_size).prefetch(1)

tf_ds = window_dataset(X_train, y_train, X_window_size, y_window_size, stride, shift, batch_size)

for x, y in tf_ds.take(1):
    print(x.shape, y.shape)
    print(x)
    print(y)
    break

(1, 7, 13) (1, 7, 1)
tf.Tensor(
[[[ 9.96500e+02  2.65380e+02 -8.78000e+00  9.44000e+01  3.33000e+00
    3.14000e+00  1.90000e-01  1.96000e+00  3.15000e+00  1.30786e+03
    2.10000e-01  6.30000e-01  1.92700e+02]
  [ 9.99340e+02  2.68770e+02 -5.19000e+00  9.44000e+01  4.39000e+00
    4.15000e+00  2.50000e-01  2.58000e+00  4.15000e+00  1.29347e+03
    2.80000e-01  1.00000e+00  1.48100e+02]
  [ 1.00239e+03  2.67700e+02 -6.35000e+00  9.21000e+01  4.12000e+00
    3.79000e+00  3.30000e-01  2.36000e+00  3.78000e+00  1.30168e+03
    2.70000e-01  1.00000e+00  2.92800e+02]
  [ 9.92190e+02  2.72350e+02 -5.61000e+00  7.30000e+01  5.50000e+00
    4.02000e+00  1.49000e+00  2.52000e+00  4.05000e+00  1.27001e+03
    2.77000e+00  4.75000e+00  2.32100e+02]
  [ 9.84740e+02  2.72790e+02 -2.58000e+00  9.28000e+01  5.44000e+00
    5.05000e+00  3.90000e-01  3.20000e+00  5.13000e+00  1.26066e+03
    2.55000e+00  4.75000e+00  2.45300e+02]
  [ 9.97710e+02  2.59990e+02 -1.54600e+01  8.39000e+01  2.18000e+00
    1

In [5]:
print(tf.__version__)

1.14.0
